## 1. 모델링

사전설정 (모델 훈련과 예측 모두 공통)

In [1]:
# cols = ["KOSPI", "CR", "NASDAQ", "DOW", "NIKKEI", "SHANGHAI", "INDI", "FOREIGN", "ORG"]
cols = ["KOSPI", "CR", "NASDAQ", "DOW", "NIKKEI", "SHANGHAI"]
len_x_ARMA = 10
len_y_nextday = 20

1-1) 데이터 크롤링

In [2]:
from kospi_predict import Crawler

crawler = Crawler(crawl_page_max=50, perPage=100)
# crawler.crawlData(cols, save=True)
crawler.loadFromSavedFile(cols)
df_crawled = crawler.removeNan()
df_crawled

KeyError: None

1-2. 데이터 전처리

In [3]:
from kospi_predict import DataPreprocessor

dpp = DataPreprocessor(df_crawled, cols)
# dpp.removeOutlier()
dpp.sortByDate()
dpp.scalingForModeling()
dpp.makeTargetXs(len_x_ARMA)
dpp.makeTargetYs(len_y_nextday)
dpp.cutoffData(len_x_ARMA, len_y_nextday)
dpp.splitData()
dpp.df_cutoff

,date,KOSPI,CR,NASDAQ,DOW,NIKKEI,SHANGHAI,INDI,FOREIGN,ORG,...,Y_KOSPI_scaled_nextday_11,Y_KOSPI_scaled_nextday_12,Y_KOSPI_scaled_nextday_13,Y_KOSPI_scaled_nextday_14,Y_KOSPI_scaled_nextday_15,Y_KOSPI_scaled_nextday_16,Y_KOSPI_scaled_nextday_17,Y_KOSPI_scaled_nextday_18,Y_KOSPI_scaled_nextday_19,Y_KOSPI_scaled_nextday_20
0,2014-04-11,1997.44,1036.0,3999.734,16026.75,13960.05,232.155,-1.727643e+10,-3.990578e+09,2.724898e+10,...,0.276915,0.274485,0.272872,0.271600,0.261013,0.266815,0.270036,0.274577,0.284314,0.299415
1,2014-04-14,1997.02,1040.7,4022.694,16173.24,13910.16,231.780,-2.301249e+10,-9.603247e+09,3.924425e+10,...,0.274485,0.272872,0.271600,0.261013,0.266815,0.270036,0.274577,0.284314,0.299415,0.299074
2,2014-04-15,1992.27,1041.5,4034.161,16262.56,13996.81,230.917,2.909747e+10,-3.820040e+10,7.678817e+09,...,0.272872,0.271600,0.261013,0.266815,0.270036,0.274577,0.284314,0.299415,0.299074,0.300828
3,2014-04-16,1992.21,1039.0,4086.225,16424.85,14417.68,230.313,5.851080e+10,4.462989e+10,-1.033886e+11,...,0.271600,0.261013,0.266815,0.270036,0.274577,0.284314,0.299415,0.299074,0.300828,0.301748
4,2014-04-17,1992.05,1040.0,4095.516,16408.54,14417.53,230.924,-1.620758e+09,-2.195242e+10,2.557773e+10,...,0.261013,0.266815,0.270036,0.274577,0.284314,0.299415,0.299074,0.300828,0.301748,0.299648
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1908,2022-01-14,2921.92,1188.0,14893.753,35911.81,28124.28,286.838,8.069229e+11,-2.451263e+11,-5.953139e+11,...,0.676662,0.699632,0.696818,0.697581,0.709694,0.711361,0.698252,0.674854,0.659731,0.688494
1909,2022-01-17,2890.10,1192.5,14700.325,35640.14,28333.52,287.162,4.814124e+11,-6.888309e+11,1.781832e+11,...,0.699632,0.696818,0.697581,0.709694,0.711361,0.698252,0.674854,0.659731,0.688494,0.696293
1910,2022-01-18,2864.24,1192.5,14506.897,35368.47,28257.25,287.007,1.999450e+11,1.569983e+10,-2.274035e+11,...,0.696818,0.697581,0.709694,0.711361,0.698252,0.674854,0.659731,0.688494,0.696293,0.696526
1911,2022-01-19,2842.28,1191.0,14340.255,35028.65,27467.23,286.982,-2.936665e+10,3.468124e+10,-1.191729e+10,...,0.697581,0.709694,0.711361,0.698252,0.674854,0.659731,0.688494,0.696293,0.696526,0.696136


1-3) 모델 훈련하고 생성

In [4]:
from kospi_predict import ModelMaker

maker = ModelMaker(dpp.y_list, dpp.df_train, dpp.df_test)
maker.makeModel(EPOCHS=800, history_plot_cutoff=30, learning_rate=0.0008)
maker.validateModel(y_no=0)

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 180)               0         
                                                                 
 dense (Dense)               (None, 1260)              228060    
                                                                 
 dense_1 (Dense)             (None, 1260)              1588860   
                                                                 
 dense_2 (Dense)             (None, 1260)              1588860   
                                                                 
 dense_3 (Dense)             (None, 1260)              1588860   
                                                                 
 dense_4 (Dense)             (None, 1260)              1588860   
                                             

2022-02-24 14:18:00.365886: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-02-24 14:18:00.365970: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-02-24 14:18:00.462781: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-02-24 14:18:00.753393: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-24 14:18:02.296662: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


KeyboardInterrupt: 

## 2. 최근 데이터로 20일 간의 KOSPI 지수 예측

2-1) 데이터 수집

In [ ]:
from kospi_predict import Crawler

#예측을 위해서는 현재기준 이전 10일간의 데이터만 있으면 됨 (len_x_ARMA가 10일때)
crawler_pred = Crawler(crawl_page_max=1, perPage=50) #이전 추세도 볼겸 넉넉히 수집
crawler_pred.crawlData(cols)
df_crawled_pred = crawler_pred.removeNan()
df_crawled_pred

2-2) 데이터 전처리

In [ ]:
from kospi_predict import DataPreprocessor

#아래는 예측을 위한 전처리
dpp_pred = DataPreprocessor(df_crawled_pred, cols)
dpp_pred.sortByDate()
dpp_pred.scalingForPredict()
dpp_pred.makeTargetXs(len_x_ARMA)
dpp_pred.cutoffData(len_x_ARMA, 0)

2-3) 예측

In [ ]:
from kospi_predict import Predictor

#2.18 기준으로 20일 뒤까지 예측하고 시각화

predictor = Predictor(dpp_pred.df)
predictor.predict()
predictor.showPredictionPlot()

In [ ]:

# predictor.saveModelToJS()